In [ ]:
!pip install --upgrade tpot

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from tpot import TPOTRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt


In [2]:
# --- Data Collection and Preparation ---
df = pd.read_csv('../power_predict/data/merged_dataset.csv') 
df.head(5)

,Unnamed: 0,Month_year,Country,Balance,Combustible Renewables,Hydro,Other Renewables,Solar,"Total Renewables (Hydro, Geo, Solar, Wind, Other)",Wind,value_CDD_18,value_CDD_21,value_Global_Horizontal_Irrandiance,value_HDD_16,value_HDD_18,value_Heat_index,value_Relative_Humidty,value_Temperature,value_Total_Precipitation,"total target (wind, solar, hydro)"
0,0,2010-04-01,Australia,Net Electricity Production,216.287,1044.406,0.00,26.811,1638.098,350.511,48.13000,12.620000,720994.0,17.47,37.08,18.410,57.37,23.210,0.05483,1421.728
1,1,2010-04-01,Austria,Net Electricity Production,350.383,2504.130,0.00,9.965,2995.696,131.107,0.05449,0.000000,736161.0,228.90,287.80,8.410,69.85,6.748,0.10510,2645.202
2,2,2010-04-01,Belgium,Net Electricity Production,383.177,141.898,0.00,80.419,702.509,97.015,0.99160,0.000000,723195.0,195.80,253.70,9.577,67.90,9.322,0.03999,319.332
3,3,2010-04-01,Canada,Net Electricity Production,712.988,28243.738,2.44,24.184,29709.301,725.951,0.06883,0.000023,679927.0,246.70,306.20,7.797,70.85,-1.708,0.05456,28993.873
4,4,2010-04-01,Chile,Net Electricity Production,152.826,1740.416,0.00,0.000,1913.601,20.359,1.41100,0.040190,578355.0,113.00,168.30,12.440,66.58,9.672,0.15780,1760.775


In [3]:
# df.columns

Index(['Unnamed: 0', 'Month_year', 'Country', 'Balance',
       'Combustible Renewables', 'Hydro', 'Other Renewables', 'Solar',
       'Total Renewables (Hydro, Geo, Solar, Wind, Other)', 'Wind',
       'value_CDD_18', 'value_CDD_21', 'value_Global_Horizontal_Irrandiance',
       'value_HDD_16', 'value_HDD_18', 'value_Heat_index',
       'value_Relative_Humidty', 'value_Temperature',
       'value_Total_Precipitation', 'total target (wind, solar, hydro)'],
      dtype='object')

In [4]:
# df.dtypes

Unnamed: 0                                             int64
Month_year                                            object
Country                                               object
Balance                                               object
Combustible Renewables                               float64
Hydro                                                float64
Other Renewables                                     float64
Solar                                                float64
Total Renewables (Hydro, Geo, Solar, Wind, Other)    float64
Wind                                                 float64
value_CDD_18                                         float64
value_CDD_21                                         float64
value_Global_Horizontal_Irrandiance                  float64
value_HDD_16                                         float64
value_HDD_18                                         float64
value_Heat_index                                     float64
value_Relative_Humidty  

## --- Data Preprocessing ---

In [5]:
# --- Data Preprocessing ---

# Setting Country + Month year as Index
df['Country_Month'] = df['Country'] + '_' + df['Month_year'].astype(str)
df = df.set_index('Country_Month')

# Separating features and target variables
X = df.drop(['Unnamed: 0', 'Month_year', 'Balance',
       'Combustible Renewables', 'Hydro', 'Other Renewables', 'Solar',
       'Total Renewables (Hydro, Geo, Solar, Wind, Other)', 'Wind',
       'total target (wind, solar, hydro)'], axis=1) 
y = df['total target (wind, solar, hydro)'].values.ravel()  ### TPOT can only handle one target at a time, expects 1D array!

# y = df[['Solar', 'Hydro', 'Wind','total target (wind, solar, hydro)']] 

In [ ]:
# Init list of numerical columns
num_features = X.select_dtypes(include=[np.number]).columns.tolist()

# Create a preprocessing pipeline
preprocessing_pipeline = ColumnTransformer(
    transformers=[
        # Preprocessing for numerical features
        ('num', Pipeline(steps=[
            ('scaler', MinMaxScaler())  # Scale features using MinMaxScaler
        ]), num_features),
        
        # Preprocessing for categorical feature 'country'
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['Country'])
    ])

preprocessing_pipeline

In [ ]:
# X

In [ ]:
# y

In [ ]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Fit the preprocessing pipeline on the training data
X_train_processed = preprocessing_pipeline.fit_transform(X_train)

# Transform the testing data using the fitted pipeline
X_test_processed = preprocessing_pipeline.transform(X_test)

## --- Model Building with TPOT ---

In [ ]:
# --- Model Building with TPOT ---
# TPOT regressor for multi-target regression
tpot_regressor = TPOTRegressor(generations=5, population_size=50, verbosity=2, random_state=42,
                               config_dict='TPOT sparse', cv=5)

# Pipeline including preprocessing and TPOT regressor
pipeline = Pipeline(steps=[('preprocessor', preprocessing_pipeline),
                           ('tpot_regressor', tpot_regressor)])

pipeline

In [ ]:
# --- Model Training ---
# Train the TPOT model
pipeline.fit(X_train, y_train)

In [ ]:
# --- Model Evaluation ---
# Evaluate the best model found by TPOT
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Output performance metrics
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

# Feature importance (TPOT might select a model where feature importance is not available)
# Here, we're assuming TPOT selected a Random Forest model. If not, this part would need to be adjusted.
if 'feature_importances_' in dir(pipeline.named_steps['tpot_regressor'].fitted_pipeline_[-1]):
    feature_importances = pipeline.named_steps['tpot_regressor'].fitted_pipeline_[-1].feature_importances_

    # Create a bar chart of feature importances
    plt.barh(range(len(feature_importances)), feature_importances, align='center')
    plt.yticks(range(len(feature_importances)), features.columns)
    plt.xlabel('Feature Importance')
    plt.ylabel('Feature')
    plt.show()
else:
    print("Selected model does not support feature importances.")